<a href="https://colab.research.google.com/github/MAHA06/Adversarials/blob/master/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount(‘/content/gdrive’)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
np.random.seed(42)
torch.manual_seed(42)
import pickle


In [ ]:
train_batch,test_batch=128,128
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0,), (1.0,))])
dataset = datasets.MNIST(root = './data', train=True, transform = transform, download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])
test_set = datasets.MNIST(root = './data', train=False, transform = transform, download=True)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=1,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=1,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=1,shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

In [ ]:
def JacobianMatrix(model,input):
  J = torch.zeros ((1, 784, 10))   # loop will fill in Jacobian
  input.requires_grad = True
  preds = model (input)
  for  i in range (10):
      grd = torch.zeros ((1, 10))   # same shape as preds
      grd[0, i] = 1    # column of Jacobian to compute
      preds.backward (gradient = grd, retain_graph = True)
      J[:,:,i] = input.grad   # fill in one column of Jacobian
      input.grad.zero_()   # .backward() accumulates gradients, so reset to zero


In [ ]:
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [ ]:
def fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,epochs):
  data_loader={'train':train_loader,'val':val_loader}
  print("Starting to fit the model")
  train_error,test_error=[],[]
  for epc in range(epochs):
    loss_epoch,val_loss_epoch=0,0
    for phase in ('train','val'):
      for i,data in enumerate(data_loader[phase]):
        input,label=data[0].to(device),data[1].to(device)
        out=model(input)
        loss=criterion(out,label)
        if phase == 'train':
          optimizer.zero_grad()

          loss.backward()
          optimizer.step()
          loss_epoch+=loss.item()
        else:
          val_loss_epoch+=loss.item()
    scheduler.step(val_loss_epoch/len(val_loader))
    print("Epoch : {} Train Loss : {} Eval Loss : {}".format(epc+1,loss_epoch/len(train_loader),val_loss_epoch/len(val_loader)))
    train_error.append(loss_epoch)
    test_error.append(val_loss_epoch)
  return train_error,test_error
        
          


In [ ]:
model=Net().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


###############
loss,val_loss=fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,10)


Starting to fit the model
Epoch : 1 Train Loss : 0.28651975764641036 Eval Loss : 0.14627669881854158
Epoch : 2 Train Loss : 0.11347357886673946 Eval Loss : 0.10931144240655485
Epoch : 3 Train Loss : 0.08692437338439123 Eval Loss : 0.09825048338787129
Epoch : 4 Train Loss : 0.0750573268817064 Eval Loss : 0.09482430201677146
Epoch : 5 Train Loss : 0.06952555202590949 Eval Loss : 0.09283972438205164
Epoch : 6 Train Loss : 0.06409916755790591 Eval Loss : 0.09230325015474836
Epoch : 7 Train Loss : 0.06028843690279511 Eval Loss : 0.0955202235781442
Epoch : 8 Train Loss : 0.0595200856316896 Eval Loss : 0.08478781001088999
Epoch : 9 Train Loss : 0.05648008505147254 Eval Loss : 0.08112028795381171
Epoch : 10 Train Loss : 0.054520659658033745 Eval Loss : 0.09514656835246962


In [ ]:
import pickle
filename = 'model_MNIST'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def fgsm_attack(input,epsilon,data_grad):
  pert_out = input + epsilon*data_grad.sign()
  pert_out = torch.clamp(pert_out, 0, 1)
  return pert_out

def ifgsm_attack(input,epsilon,data_grad):
  iter = 10
  alpha = epsilon/iter
  pert_out = input
  for i in range(iter-1):
    pert_out = pert_out + alpha*data_grad.sign()
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

def mifgsm_attack(input,epsilon,data_grad):
  iter=10
  decay_factor=1.0
  pert_out = input
  alpha = epsilon/iter
  g=0
  for i in range(iter-1):
    g = decay_factor*g + data_grad/torch.norm(data_grad,p=1)
    pert_out = pert_out + alpha*torch.sign(g)
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

In [ ]:
filename = 'model_MNIST'

model= pickle.load(open(filename, 'rb'))
def testModel(model,criterion,device,test_loader,attack):
  for data,labels in test_loader:
    data,labels=data.to(device),labels.to(device)
    data.requires_grad=true
    outs=model(data)
    true_preds=outs.argmax(outs,dim=1)
    if true_preds!=labels:
      continue
    true_loss=criterion(outs,labels)
    if attack == "fgsm":
        perturbed_data = fgsm_attack(data,epsilon,data_grad)
      elif attack == "ifgsm":
        perturbed_data = ifgsm_attack(data,epsilon,data_grad)
      elif attack == "mifgsm":
        perturbed_data = mifgsm_attack(data,epsilon,data_grad)
      elif attack == "jacobian":
        print("Implement the jacobian")
    p_out=model(perturbed_data)
    p_prediction=p_out.
